In [1]:
#install & imports
!pip install -q librosa soundfile tqdm

import os
import glob
import numpy as np
import librosa
import librosa.effects as effects
import soundfile as sf
from tqdm import tqdm

# sample rate for loading/saving
TARGET_SR = 16000
print("Libraries loaded. TARGET_SR =", TARGET_SR)

Libraries loaded. TARGET_SR = 16000


In [2]:
#set paths
BASE_MYDRIVE = "/content/drive/MyDrive"
IRMAS_MELS = os.path.join(BASE_MYDRIVE, "irmas_mels")
OUTPUT_DIR = os.path.join(BASE_MYDRIVE, "irmas", "augmentations")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# instruments,exactly one file from (first file in each folder)
instrument_list = ["cel", "cla", "flu"]

print("IRMAS_MELS:", IRMAS_MELS)
print("OUTPUT_DIR:", OUTPUT_DIR)
print("Instruments:", instrument_list)

IRMAS_MELS: /content/drive/MyDrive/irmas_mels
OUTPUT_DIR: /content/drive/MyDrive/irmas/augmentations
Instruments: ['cel', 'cla', 'flu']


In [3]:
#augmentation functions
def augment_pitch_shift(y, sr, n_steps=2):
    return effects.pitch_shift(y=y, sr=sr, n_steps=n_steps)

def augment_time_stretch(y, rate=0.85):
    return effects.time_stretch(y=y, rate=rate)

def augment_add_noise(y, noise_factor=0.005):
    noise = np.random.randn(len(y))
    y_aug = y + noise_factor * noise
    # avoid clipping: normalize to 95% of full scale
    maxv = np.max(np.abs(y_aug))
    if maxv > 0:
        y_aug = y_aug / maxv * 0.95
    return y_aug

In [4]:
# select one file (first) from each instrument's preprocessed_audio folder
selected_files = {}   # instrument -> file path

for inst in instrument_list:
    pre_dir = os.path.join(IRMAS_MELS, inst, "preprocessed_audio")
    if not os.path.isdir(pre_dir):
        print(f"WARNING: {inst} preprocessed_audio folder NOT FOUND at: {pre_dir}")
        continue

    files = []
    for ext in ("*.wav", "*.mp3", "*.flac", "*.ogg"):
        files.extend(glob.glob(os.path.join(pre_dir, ext)))
    files = sorted(files)

    if len(files) == 0:
        print(f"WARNING: no audio files found for {inst} in {pre_dir}")
        continue

    # pick the first file found
    chosen = files[0]
    selected_files[inst] = chosen
    print(f"{inst}: selected {os.path.basename(chosen)}")

if len(selected_files) == 0:
    raise RuntimeError("No files selected. Check your IRMAS_MELS path and that preprocessed_audio folders exist.")

cel: selected 008__[cel][nod][cla]0058__1_preprocessed.wav
cla: selected 004__[cla][nod][cla]0242__1_preprocessed.wav
flu: selected 008__[flu][nod][cla]0393__1_preprocessed.wav


In [5]:
#apply augmentations & save outputs
PITCH_STEPS = +2     # semitones
TIME_RATE = 0.85     # <1 slower, >1 faster
NOISE_FACTOR = 0.005

for inst, src in selected_files.items():
    fname = os.path.splitext(os.path.basename(src))[0]
    inst_out = os.path.join(OUTPUT_DIR, inst, fname)
    os.makedirs(inst_out, exist_ok=True)

    # load audio (resample to TARGET_SR)
    y, sr = librosa.load(src, sr=TARGET_SR, mono=True)

    # small normalization of source
    if np.max(np.abs(y)) > 0:
        y = y / np.max(np.abs(y))

    # 1) Pitch shift
    try:
        y_ps = augment_pitch_shift(y, sr=sr, n_steps=PITCH_STEPS)
        out_ps = os.path.join(inst_out, f"{fname}_pitch{PITCH_STEPS:+d}st.wav")
        sf.write(out_ps, y_ps, sr)
    except Exception as e:
        print(f"{inst} - pitch shift failed for {fname}: {e}")

    # 2) Time stretch
    try:
        y_ts = augment_time_stretch(y, rate=TIME_RATE)
        out_ts = os.path.join(inst_out, f"{fname}_stretch{TIME_RATE:.2f}.wav")
        sf.write(out_ts, y_ts, sr)
    except Exception as e:
        print(f"{inst} - time stretch failed for {fname}: {e}")

    # 3) Additive noise
    try:
        y_n = augment_add_noise(y, noise_factor=NOISE_FACTOR)
        out_n = os.path.join(inst_out, f"{fname}_noise{NOISE_FACTOR:.4f}.wav")
        sf.write(out_n, y_n, sr)
    except Exception as e:
        print(f"{inst} - add noise failed for {fname}: {e}")

    print(f"Done: {inst} -> {fname} (saved augmentations if no errors)")

Done: cel -> 008__[cel][nod][cla]0058__1_preprocessed (saved augmentations if no errors)
Done: cla -> 004__[cla][nod][cla]0242__1_preprocessed (saved augmentations if no errors)
Done: flu -> 008__[flu][nod][cla]0393__1_preprocessed (saved augmentations if no errors)


In [7]:
#list saved outputs
print("\nAugmented output root:", OUTPUT_DIR)
for inst in instrument_list:
    inst_dir = os.path.join(OUTPUT_DIR, inst)
    if not os.path.isdir(inst_dir):
        print(f"{inst}: (no output folder)")
        continue

    print("\nInstrument:", inst)
    for sample in sorted(os.listdir(inst_dir)):
        sample_dir = os.path.join(inst_dir, sample)
        if os.path.isdir(sample_dir):
            wavs = [f for f in sorted(os.listdir(sample_dir)) if f.lower().endswith('.wav')]
            print(" ", sample, "->", len(wavs), "files")
            for w in wavs:
                print("    ", w)


Augmented output root: /content/drive/MyDrive/irmas/augmentations

Instrument: cel
  008__[cel][nod][cla]0058__1_preprocessed -> 3 files
     008__[cel][nod][cla]0058__1_preprocessed_noise0.0050.wav
     008__[cel][nod][cla]0058__1_preprocessed_pitch+2st.wav
     008__[cel][nod][cla]0058__1_preprocessed_stretch0.85.wav

Instrument: cla
  004__[cla][nod][cla]0242__1_preprocessed -> 3 files
     004__[cla][nod][cla]0242__1_preprocessed_noise0.0050.wav
     004__[cla][nod][cla]0242__1_preprocessed_pitch+2st.wav
     004__[cla][nod][cla]0242__1_preprocessed_stretch0.85.wav

Instrument: flu
  008__[flu][nod][cla]0393__1_preprocessed -> 3 files
     008__[flu][nod][cla]0393__1_preprocessed_noise0.0050.wav
     008__[flu][nod][cla]0393__1_preprocessed_pitch+2st.wav
     008__[flu][nod][cla]0393__1_preprocessed_stretch0.85.wav
